# Import Library

In [2]:
import csv, os

# ==============================================================================
# Data manipulation
# ==============================================================================
import numpy as np
import pandas as pd

# ==============================================================================
# Plots
# ==============================================================================
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
plt.rcParams['lines.linewidth'] = 1.5
%matplotlib inline

# ==============================================================================
# Modeling and Forecasting
# ==============================================================================
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# !pip install skforecast
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom
from skforecast.ForecasterAutoregDirect import ForecasterAutoregDirect
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster
from skforecast.utils import save_forecaster
from skforecast.utils import load_forecaster

# ==============================================================================
# Warnings configuration
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')

     |████████████████████████████████| 112 kB 36.9 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=259c959d2a4f0b0a060ca322986feec147eaccbe4356d436dd313b8cc824a8e5
  Stored in directory: /root/.cache/pip/wheels/7f/1a/65/84ff8c386bec21fca6d220ea1f5498a0367883a78dd5ba6122
Successfully built pyperclip


# Import Data

In [3]:
df = pd.read_csv('UN_data_country_specific_till_20250.csv')
df.head()

,LocTypeID,LocTypeName,Location,Time,TPopulation1July,TPopulationMale1July,TPopulationFemale1July,PopDensity,PopSexRatio,MedianAgePop,...,LE65Male,LE65Female,LE80,LE80Male,LE80Female,InfantDeaths,IMR,LBsurvivingAge1,Under5Deaths,NetMigrations
0,4,Country/Area,Afghanistan,1960,8622.466,4476.521,4145.945,13.2797,107.9735,17.9998,...,8.8874,9.5773,4.0539,3.9313,4.1994,103.790,240.5329,364.106,151.545,2.606
1,4,Country/Area,Afghanistan,1961,8790.140,4556.369,4233.771,13.5380,107.6196,17.9244,...,8.9651,9.6573,4.0900,3.9663,4.2350,104.080,236.1826,373.167,152.016,6.109
2,4,Country/Area,Afghanistan,1962,8969.046,4642.166,4326.881,13.8135,107.2866,17.8525,...,9.0281,9.7308,4.1211,3.9945,4.2681,104.617,232.1626,382.932,152.887,7.016
3,4,Country/Area,Afghanistan,1963,9157.464,4732.954,4424.510,14.1037,106.9712,17.7876,...,9.0874,9.8036,4.1511,4.0207,4.3010,105.263,228.2390,393.236,153.919,6.681
4,4,Country/Area,Afghanistan,1964,9355.514,4828.822,4526.692,14.4087,106.6744,17.7305,...,9.1502,9.8749,4.1812,4.0481,4.3326,105.942,224.3317,403.990,154.971,7.079


# Data Preprocessing
- Drop useless columns like LocTypeID, LocTypeName, and Location
- Fill missing data with the median

In [4]:
df = pd.read_csv('UN_data_country_specific_till_20250.csv')
df = df.drop(['LocTypeID','LocTypeName', 'Location'], axis = 1)

to_save_df = df.copy()

# Fill mean of NA with group Location(Country)
# df = df.groupby('Location').transform(lambda x: x.fillna(x.mean()))
# df = df.drop(['Location' ], axis = 1)

# Fill NA with global median
for col in df.columns:
  df[col] = df[col].fillna(df[col].median())

df.head()

,Time,TPopulation1July,TPopulationMale1July,TPopulationFemale1July,PopDensity,PopSexRatio,MedianAgePop,NatChange,NatChangeRT,PopChange,...,LE65Male,LE65Female,LE80,LE80Male,LE80Female,InfantDeaths,IMR,LBsurvivingAge1,Under5Deaths,NetMigrations
0,1960,8622.466,4476.521,4145.945,13.2797,107.9735,17.9998,158.818,18.419,161.436,...,8.8874,9.5773,4.0539,3.9313,4.1994,103.790,240.5329,364.106,151.545,2.606
1,1961,8790.140,4556.369,4233.771,13.5380,107.6196,17.9244,167.811,19.094,173.912,...,8.9651,9.6573,4.0900,3.9663,4.2350,104.080,236.1826,373.167,152.016,6.109
2,1962,8969.046,4642.166,4326.881,13.8135,107.2866,17.8525,176.875,19.725,183.901,...,9.0281,9.7308,4.1211,3.9945,4.2681,104.617,232.1626,382.932,152.887,7.016
3,1963,9157.464,4732.954,4424.510,14.1037,106.9712,17.7876,186.264,20.344,192.935,...,9.0874,9.8036,4.1511,4.0207,4.3010,105.263,228.2390,393.236,153.919,6.681
4,1964,9355.514,4828.822,4526.692,14.4087,106.6744,17.7305,196.084,20.964,203.164,...,9.1502,9.8749,4.1812,4.0481,4.3326,105.942,224.3317,403.990,154.971,7.079


Split data for training, validaion, and test

In [5]:
train_year = 2020
df_train, df_test = df[df.Time <=train_year], df[df.Time>train_year]
df_train.head()

,Time,TPopulation1July,TPopulationMale1July,TPopulationFemale1July,PopDensity,PopSexRatio,MedianAgePop,NatChange,NatChangeRT,PopChange,...,LE65Male,LE65Female,LE80,LE80Male,LE80Female,InfantDeaths,IMR,LBsurvivingAge1,Under5Deaths,NetMigrations
0,1960,8622.466,4476.521,4145.945,13.2797,107.9735,17.9998,158.818,18.419,161.436,...,8.8874,9.5773,4.0539,3.9313,4.1994,103.790,240.5329,364.106,151.545,2.606
1,1961,8790.140,4556.369,4233.771,13.5380,107.6196,17.9244,167.811,19.094,173.912,...,8.9651,9.6573,4.0900,3.9663,4.2350,104.080,236.1826,373.167,152.016,6.109
2,1962,8969.046,4642.166,4326.881,13.8135,107.2866,17.8525,176.875,19.725,183.901,...,9.0281,9.7308,4.1211,3.9945,4.2681,104.617,232.1626,382.932,152.887,7.016
3,1963,9157.464,4732.954,4424.510,14.1037,106.9712,17.7876,186.264,20.344,192.935,...,9.0874,9.8036,4.1511,4.0207,4.3010,105.263,228.2390,393.236,153.919,6.681
4,1964,9355.514,4828.822,4526.692,14.4087,106.6744,17.7305,196.084,20.964,203.164,...,9.1502,9.8749,4.1812,4.0481,4.3326,105.942,224.3317,403.990,154.971,7.079


In [6]:
df_toSave = df_test.copy()

In [7]:
y_train = df_train[['NetMigrations']]
X_train = df_train.drop(['NetMigrations'], axis = 1)

y_test = df_test[['NetMigrations']]
X_test = df_test.drop(['NetMigrations'], axis = 1)

# Model Training & Validation

In [8]:
import sklearn.metrics as metrics
def regression_results(y_true, y_pred):    # Regression metrics
    metrics_dict = {}

    try:
      explained_variance=metrics.explained_variance_score(y_true, y_pred)
      # print('explained_variance: ', round(explained_variance,4))       
      metrics_dict['explained_variance'] = round(explained_variance,4)
    except: pass
    
    try:
      mean_absolute_error=metrics.mean_absolute_error(y_true, y_pred) 
      mse=metrics.mean_squared_error(y_true, y_pred) 
      # print('MSE: ', round(mse,4))
      # print('RMSE: ', round(np.sqrt(mse),4))  
      metrics_dict['MSE'] = round(mse,4)
      metrics_dict['RMSE'] = round(np.sqrt(mse),4)
    except: pass
    
    try:
      mean_squared_log_error=metrics.mean_squared_log_error(y_true, y_pred)
      # print('mean_squared_log_error: ', round(mean_squared_log_error,4))  
      metrics_dict['mean_squared_log_error'] = mean_squared_log_error
    except: pass
    
    try:
      median_absolute_error=metrics.median_absolute_error(y_true, y_pred)
      # print('MAE: ', round(mean_absolute_error,4))  
      metrics_dict['MAE'] = round(mean_absolute_error,4)
    except: pass
    
    try:
      r2=metrics.r2_score(y_true, y_pred)
      # print('r2: ', round(r2,4))  
      metrics_dict['r2'] = round(r2,4)
    except: pass
    
    return metrics_dict

# Grid Searching Hyperparameters
The best model is **RandomForestRegressor(max_depth=5, max_features='log2')**

In [9]:
# Spot Check Algorithms
from sklearn.model_selection import TimeSeriesSplit
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score

models = []
models.append(('LR', LinearRegression()))
models.append(('NN', MLPRegressor(solver = 'lbfgs')))  #neural network
models.append(('KNN', KNeighborsRegressor())) 
models.append(('RF', RandomForestRegressor(n_estimators = 10))) # Ensemble method - collection of many decision trees
models.append(('SVR', SVR(gamma='auto'))) # kernel = linear# Evaluate each model in turn

In [10]:
#  custom scorer
from sklearn.metrics import make_scorer

def rmse(actual, predict):
  predict = np.array(predict)
  actual = np.array(actual)
  distance = predict - actual
  square_distance = distance ** 2
  mean_square_distance = square_distance.mean()
  score = np.sqrt(mean_square_distance)
  return score

rmse_score = make_scorer(rmse, greater_is_better = False)

In [11]:
from sklearn.model_selection import GridSearchCV

model = RandomForestRegressor()
param_search = {'n_estimators': [20, 50, 100],
          'max_features': ['auto', 'sqrt', 'log2'],
          'max_depth' : [i for i in range(5,10)]
}

gsearch = GridSearchCV(estimator=model, 
            cv = TimeSeriesSplit(n_splits=12),
            param_grid=param_search, 
            scoring = rmse_score)

gsearch.fit(X_train, y_train) 

best_score = gsearch.best_score_
print(f'Score for the best model = {best_score}')

best_model = gsearch.best_estimator_
best_model

Score for the best model = -134.15929246561083


RandomForestRegressor(max_depth=5, max_features='sqrt')

# Prediction
With the best model, the MSE for test is 7378.2331

In [42]:
# Checking best model performance on test data
y_true = y_train.values
y_pred = best_model.predict(X_train)
my_metrics = regression_results(y_true, y_pred)
print(f'Eval MSE = {my_metrics['MSE']}')
print(f'Eval RMSE = {my_metrics['RMSE']}')

y_true = y_test.values
y_pred = best_model.predict(X_test)
my_metrics = regression_results(y_true, y_pred)
print(f'Test MSE = {my_metrics['MSE']}')
print(f'Test RMSE = {my_metrics['RMSE']}')

Eval MSE = 7691.1846
Eval RMSE = 87.6994

Test MSE = 16478.0906
Test RMSE = 128.367


# Save prediction result

In [44]:
y_pred = best_model.predict(X_train)
df = pd.read_csv('UN_data_country_specific_till_20250.csv')
df_train_val_toSave = df[df.Time<=train_year][['Location', 'Time', 'NetMigrations']]
df_train_val_toSave = df_train_val_toSave.rename({'Location': 'Country', 'NetMigrations': 'Actual_Net_Migration'}, axis=1)
df_train_val_toSave.reset_index()
df_train_val_toSave['Predicted_Net_Migration'] = y_pred
df_train_val_toSave.to_csv('Random_Forest_Predicted_2010_2020.csv')

df_train_val_toSave

,Country,Time,Actual_Net_Migration,Predicted_Net_Migration
0,Afghanistan,1960,2.606,-11.104308
1,Afghanistan,1961,6.109,-11.104308
2,Afghanistan,1962,7.016,-11.104308
3,Afghanistan,1963,6.681,-8.376197
4,Afghanistan,1964,7.079,-8.376197
...,...,...,...,...
19530,Zimbabwe,2016,-59.918,-6.456920
19531,Zimbabwe,2017,-59.918,-6.456920
19532,Zimbabwe,2018,-59.918,-6.258366
19533,Zimbabwe,2019,-59.918,-6.456920
